In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.6 MB/s eta 0:00:00


In [ ]:
# importing necessary libaries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from catboost import CatBoostClassifier
import joblib

In [ ]:
# loading the dataset

df = pd.read_csv('DEALS PREDICT_NEW_check.csv')

## Exploratory Data Analysis

In [ ]:
df.head()

,S/N,DATE,NAME_OF_ENTITY,NATURE_OF_BUSINESS,BUSINESS_PROJECT,CONTACT_NAME_PHONE_EMAIL_OF_PROMOTER,PROPOSED_FACILITY_AMOUNT,PURPOSE_OF_FACILITY,VALUE_CHAIN_CATEGORY,NAME_OF_BANK,...,SECURITY_PROPOSED,FEASIBILITY_STUDY_AVAILABLE,HIGHLIGHTS_OF_DISCUSSION,RM_BM_NAME_PHONE_NUMBER,RM_BM_EMAIL,STATUS_UPDATE,CHALLENGES,PROPOSED_NEXT_STEPS,STATE,TARGET
0,1,1/2/2023,COM10001,RICE PROCESSING,EXISTING,OB10001,2.000000e+09,WORKING CAPITAL ENHANCEMENT,MIDSTREAM,FIDELITY BANK PLC,...,NIL,YES,NaN,NaN,NaN,WE AREAWAITING CREDIT COMMITY DECITION ON WEAT...,NONE,NaN,ADAMAWA,1
1,2,1/2/2023,COM10002,FERTILIZER DISTRIBUTION ACROSS ADAMAWA,EXISTING,OB10002,5.000000e+07,WORKING CAPITAL ENHANCEMENT,PRE-UPSTREAM,FIDELITY BANK PLC,...,NIL,YES,NIL,NIL,NIL,WE ARE CURRENTLY WAITING FOR THE CLIENT TO CHO...,THE CLIENT IS ONLY WILLING TO DISTRIBUTE CRYST...,ONBOARD,ADAMAWA,1
2,3,1/2/2023,COM10003,BULL FEED FINISHING,EXISTING,OB10003,9.000000e+06,WORKING CAPITAL ENHANCEMENT,UPSTREAM,STANBIC IBTC,...,EQUITY CONTRIBUTION,YES,NIL,NIL,NIL,NaN,NaN,NaN,ADAMAWA,1
3,4,1/2/2023,COM10004,BULL FEED FINISHING,EXISTING,OB10004,1.000000e+05,WORKING CAPITAL ENHANCEMENT,UPSTREAM,STANBIC IBTC,...,EQUITY CONTRIBUTION,YES,NIL,NIL,NIL,NaN,NaN,NaN,ADAMAWA,0
4,5,1/2/2023,COM10005,BULL FEED FINISHING,EXISTING,OB10005,5.000000e+06,WORKING CAPITAL ENHANCEMENT,UPSTREAM,STANBIC IBTC,...,EQUITY CONTRIBUTION,YES,NIL,NIL,NIL,NaN,NaN,NaN,ADAMAWA,1


In [ ]:
df.columns


Index(['S/N', 'DATE', 'NAME_OF_ENTITY', 'NATURE_OF_BUSINESS',
       'BUSINESS_PROJECT', 'CONTACT_NAME_PHONE_EMAIL_OF_PROMOTER',
       'PROPOSED_FACILITY_AMOUNT', 'PURPOSE_OF_FACILITY',
       'VALUE_CHAIN_CATEGORY', 'NAME_OF_BANK', 'BORROWING_RELATIONSHIP',
       'FRESH_LOAN_REQUEST', 'REQUEST_SUBMITTED_TO_BANK', 'SECURITY_PROPOSED',
       'FEASIBILITY_STUDY_AVAILABLE', 'HIGHLIGHTS_OF_DISCUSSION',
       'RM_BM_NAME_PHONE_NUMBER', 'RM_BM_EMAIL', 'STATUS_UPDATE', 'CHALLENGES',
       'PROPOSED_NEXT_STEPS', 'STATE', 'TARGET'],
      dtype='object')

In [ ]:
# short statistical description of dataset

df.describe

<bound method NDFrame.describe of      S/N        DATE          NAME_OF_ENTITY  \
0      1    1/2/2023                COM10001   
1      2    1/2/2023                COM10002   
2      3    1/2/2023                COM10003   
3      4    1/2/2023                COM10004   
4      5    1/2/2023                COM10005   
..   ...         ...                     ...   
238  239  31/03/2024      NOOR NGESA NIG LTD   
239  240  30/01/2024             ABS ZUGACHI   
240  241    5/5/2024    Bauchi Meat Factory    
241  242    5/5/2024  ZAKWARAM BAFFA NIG LTD   
242  243    5/5/2024       BAUCHI FERTILIZER   

                         NATURE_OF_BUSINESS BUSINESS_PROJECT  \
0                           RICE PROCESSING         EXISTING   
1    FERTILIZER DISTRIBUTION ACROSS ADAMAWA         EXISTING   
2                       BULL FEED FINISHING         EXISTING   
3                       BULL FEED FINISHING         EXISTING   
4                       BULL FEED FINISHING         EXISTING   
..                                      ...              ...   
238                        RICE CULTIVATION         EXISTING   
239                   FERTILIZER FINANCING          EXISTING   
240                        Meat Processing          EXISTING   
241                                 POULTRY         EXISTING   
242                    FERTILIZER BLENDINFG         EXISTING   

    CONTACT_NAME_PHONE_EMAIL_OF_PROMOTER  PROPOSED_FACILITY_AMOUNT  \
0                                OB10001              2.000000e+09   
1                                OB10002              5.000000e+07   
2                                OB10003              9.000000e+06   
3                                OB10004              1.000000e+05   
4                                OB10005              5.000000e+06   
..                                   ...                       ...   
238                              OB10239              2.000000e+08   
239                              OB10240              5.000000e+08   
240                              OB10241              3.000000e+09   
241                              OB10242              2.000000e+08   
242                              OB10243              5.000000e+09   

                                   PURPOSE_OF_FACILITY VALUE_CHAIN_CATEGORY  \
0                          WORKING CAPITAL ENHANCEMENT            MIDSTREAM   
1                          WORKING CAPITAL ENHANCEMENT         PRE-UPSTREAM   
2                          WORKING CAPITAL ENHANCEMENT             UPSTREAM   
3                          WORKING CAPITAL ENHANCEMENT             UPSTREAM   
4                          WORKING CAPITAL ENHANCEMENT             UPSTREAM   
..                                                 ...                  ...   
238                     TO FINANCE CULTIVATION OF RICE             UPSTREAM   
239                               FERTILIZER PURCHASE          PRE-UPSTREAM   
240  working capital and reviving cold room and oth...            MIDSTREAM   
241                                          EXPANSION             UPSTREAM   
242                                          EXPANSION            MIDSTREAM   

          NAME_OF_BANK  ...                             SECURITY_PROPOSED  \
0    FIDELITY BANK PLC  ...                                           NIL   
1    FIDELITY BANK PLC  ...                                           NIL   
2         STANBIC IBTC  ...                           EQUITY CONTRIBUTION   
3         STANBIC IBTC  ...                           EQUITY CONTRIBUTION   
4         STANBIC IBTC  ...                           EQUITY CONTRIBUTION   
..                 ...  ...                                           ...   
238         JAIZ BANK   ...  25% EQUITY CONTRIBUTION AND NIRSAL GUARANTEE   
239           STERLING  ...                                        EQUITY   
240     Sterling Bank   ...             Factory and 50% NIRSAL guarantee    
241           STERLING  ...                      LANDED PROP

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 23 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   S/N                                   243 non-null    int64  
 1   DATE                                  243 non-null    object 
 2   NAME_OF_ENTITY                        241 non-null    object 
 3   NATURE_OF_BUSINESS                    242 non-null    object 
 4   BUSINESS_PROJECT                      243 non-null    object 
 5   CONTACT_NAME_PHONE_EMAIL_OF_PROMOTER  243 non-null    object 
 6   PROPOSED_FACILITY_AMOUNT              243 non-null    float64
 7   PURPOSE_OF_FACILITY                   240 non-null    object 
 8   VALUE_CHAIN_CATEGORY                  243 non-null    object 
 9   NAME_OF_BANK                          221 non-null    object 
 10  BORROWING_RELATIONSHIP                243 non-null    object 
 11  FRESH_LOAN_REQUEST 

In [ ]:
df.isna().sum()

S/N                                      0
DATE                                     0
NAME_OF_ENTITY                           2
NATURE_OF_BUSINESS                       1
BUSINESS_PROJECT                         0
CONTACT_NAME_PHONE_EMAIL_OF_PROMOTER     0
PROPOSED_FACILITY_AMOUNT                 0
PURPOSE_OF_FACILITY                      3
VALUE_CHAIN_CATEGORY                     0
NAME_OF_BANK                            22
BORROWING_RELATIONSHIP                   0
FRESH_LOAN_REQUEST                       0
REQUEST_SUBMITTED_TO_BANK                0
SECURITY_PROPOSED                       13
FEASIBILITY_STUDY_AVAILABLE              0
HIGHLIGHTS_OF_DISCUSSION                 1
RM_BM_NAME_PHONE_NUMBER                  1
RM_BM_EMAIL                              1
STATUS_UPDATE                           33
CHALLENGES                              73
PROPOSED_NEXT_STEPS                     43
STATE                                    3
TARGET                                   0
dtype: int6

## Feature Engineering

In [ ]:
missing_categorical_columns = []

In [ ]:
new_dict = dict(df.isna().sum())

In [ ]:
new_dict


{'S/N': 0,
 'DATE': 0,
 'NAME_OF_ENTITY': 2,
 'NATURE_OF_BUSINESS': 1,
 'BUSINESS_PROJECT': 0,
 'CONTACT_NAME_PHONE_EMAIL_OF_PROMOTER': 0,
 'PROPOSED_FACILITY_AMOUNT': 0,
 'PURPOSE_OF_FACILITY': 3,
 'VALUE_CHAIN_CATEGORY': 0,
 'NAME_OF_BANK': 22,
 'BORROWING_RELATIONSHIP': 0,
 'FRESH_LOAN_REQUEST': 0,
 'REQUEST_SUBMITTED_TO_BANK': 0,
 'SECURITY_PROPOSED': 13,
 'FEASIBILITY_STUDY_AVAILABLE': 0,
 'HIGHLIGHTS_OF_DISCUSSION': 1,
 'RM_BM_NAME_PHONE_NUMBER': 1,
 'RM_BM_EMAIL': 1,
 'STATUS_UPDATE': 33,
 'CHALLENGES': 73,
 'PROPOSED_NEXT_STEPS': 43,
 'STATE': 3,
 'TARGET': 0}

In [ ]:
# figuring out features with missing features
for k,v in new_dict.items():
    if v != 0:
        missing_categorical_columns.append(k)

In [ ]:
missing_categorical_columns

['NAME_OF_ENTITY',
 'NATURE_OF_BUSINESS',
 'PURPOSE_OF_FACILITY',
 'NAME_OF_BANK',
 'SECURITY_PROPOSED',
 'HIGHLIGHTS_OF_DISCUSSION',
 'RM_BM_NAME_PHONE_NUMBER',
 'RM_BM_EMAIL',
 'STATUS_UPDATE',
 'CHALLENGES',
 'PROPOSED_NEXT_STEPS',
 'STATE']

In [ ]:
from typing import List
def fill_categorical_columns_with_mode(col_list: List[str]) -> None:
    """ function that fills categoical columns with missing values
    with the column mode values
    """

    for col in col_list:
        mode_c = df[col].mode()[0]
        df[col].fillna(mode_c, inplace=True)
    return


fill_categorical_columns_with_mode(missing_categorical_columns)


In [ ]:
df.isna().sum()

S/N                                     0
DATE                                    0
NAME_OF_ENTITY                          0
NATURE_OF_BUSINESS                      0
BUSINESS_PROJECT                        0
CONTACT_NAME_PHONE_EMAIL_OF_PROMOTER    0
PROPOSED_FACILITY_AMOUNT                0
PURPOSE_OF_FACILITY                     0
VALUE_CHAIN_CATEGORY                    0
NAME_OF_BANK                            0
BORROWING_RELATIONSHIP                  0
FRESH_LOAN_REQUEST                      0
REQUEST_SUBMITTED_TO_BANK               0
SECURITY_PROPOSED                       0
FEASIBILITY_STUDY_AVAILABLE             0
HIGHLIGHTS_OF_DISCUSSION                0
RM_BM_NAME_PHONE_NUMBER                 0
RM_BM_EMAIL                             0
STATUS_UPDATE                           0
CHALLENGES                              0
PROPOSED_NEXT_STEPS                     0
STATE                                   0
TARGET                                  0
dtype: int64

In [ ]:
df['BUSINESS_PROJECT']

0      EXISTING
1      EXISTING
2      EXISTING
3      EXISTING
4      EXISTING
         ...   
238    EXISTING
239    EXISTING
240    EXISTING
241    EXISTING
242    EXISTING
Name: BUSINESS_PROJECT, Length: 243, dtype: object

In [ ]:
df.head()

,S/N,DATE,NAME_OF_ENTITY,NATURE_OF_BUSINESS,BUSINESS_PROJECT,CONTACT_NAME_PHONE_EMAIL_OF_PROMOTER,PROPOSED_FACILITY_AMOUNT,PURPOSE_OF_FACILITY,VALUE_CHAIN_CATEGORY,NAME_OF_BANK,...,SECURITY_PROPOSED,FEASIBILITY_STUDY_AVAILABLE,HIGHLIGHTS_OF_DISCUSSION,RM_BM_NAME_PHONE_NUMBER,RM_BM_EMAIL,STATUS_UPDATE,CHALLENGES,PROPOSED_NEXT_STEPS,STATE,TARGET
0,1,1/2/2023,COM10001,RICE PROCESSING,EXISTING,OB10001,2.000000e+09,WORKING CAPITAL ENHANCEMENT,MIDSTREAM,FIDELITY BANK PLC,...,NIL,YES,NIL,NIL,NIL,WE AREAWAITING CREDIT COMMITY DECITION ON WEAT...,NONE,AWATING FEEDBACK FROM THE PROSPECT,ADAMAWA,1
1,2,1/2/2023,COM10002,FERTILIZER DISTRIBUTION ACROSS ADAMAWA,EXISTING,OB10002,5.000000e+07,WORKING CAPITAL ENHANCEMENT,PRE-UPSTREAM,FIDELITY BANK PLC,...,NIL,YES,NIL,NIL,NIL,WE ARE CURRENTLY WAITING FOR THE CLIENT TO CHO...,THE CLIENT IS ONLY WILLING TO DISTRIBUTE CRYST...,ONBOARD,ADAMAWA,1
2,3,1/2/2023,COM10003,BULL FEED FINISHING,EXISTING,OB10003,9.000000e+06,WORKING CAPITAL ENHANCEMENT,UPSTREAM,STANBIC IBTC,...,EQUITY CONTRIBUTION,YES,NIL,NIL,NIL,DOCUMENT YET TO BE PRESENTED,NIL,AWATING FEEDBACK FROM THE PROSPECT,ADAMAWA,1
3,4,1/2/2023,COM10004,BULL FEED FINISHING,EXISTING,OB10004,1.000000e+05,WORKING CAPITAL ENHANCEMENT,UPSTREAM,STANBIC IBTC,...,EQUITY CONTRIBUTION,YES,NIL,NIL,NIL,DOCUMENT YET TO BE PRESENTED,NIL,AWATING FEEDBACK FROM THE PROSPECT,ADAMAWA,0
4,5,1/2/2023,COM10005,BULL FEED FINISHING,EXISTING,OB10005,5.000000e+06,WORKING CAPITAL ENHANCEMENT,UPSTREAM,STANBIC IBTC,...,EQUITY CONTRIBUTION,YES,NIL,NIL,NIL,DOCUMENT YET TO BE PRESENTED,NIL,AWATING FEEDBACK FROM THE PROSPECT,ADAMAWA,1


In [ ]:
# Encoding
categorical_x_features = ['BUSINESS_PROJECT', 'VALUE_CHAIN_CATEGORY',
                          'BORROWING_RELATIONSHIP', 'FRESH_LOAN_REQUEST',
                          'REQUEST_SUBMITTED_TO_BANK', 'FEASIBILITY_STUDY_AVAILABLE']

# Function to create encoder dictionaries
def encoder(feature: str) -> dict:
    """ Function that handles encoding """
    encoder_dict = {item: index for index, item in enumerate(df[feature].unique())}
    return encoder_dict

# Creating encoder dictionaries for each categorical feature
encoder_dicts = {feature: encoder(feature) for feature in categorical_x_features}

# Replace categorical values with their corresponding encodings
for feature, encoder_dict in encoder_dicts.items():
    df[feature] = df[feature].replace(encoder_dict)

In [ ]:
df['BUSINESS_PROJECT']

0      0
1      0
2      0
3      0
4      0
      ..
238    0
239    0
240    0
241    0
242    0
Name: BUSINESS_PROJECT, Length: 243, dtype: int64

In [ ]:
print(encoder_dicts)

{'BUSINESS_PROJECT': {'EXISTING': 0, 'NEW  ': 1}, 'VALUE_CHAIN_CATEGORY': {'MIDSTREAM': 0, 'PRE-UPSTREAM': 1, 'UPSTREAM': 2, 'DOWNSTREAM': 3, 'UPSTREAM AND MIDSTREAM': 4, 'MIDSTREAM AND DOWNSTREAM': 5, 'UPSTREAM AND DOWNSTREAM': 6}, 'BORROWING_RELATIONSHIP': {'YES': 0, 'NO': 1}, 'FRESH_LOAN_REQUEST': {'YES': 0, 'NO': 1}, 'REQUEST_SUBMITTED_TO_BANK': {'YES': 0, 'NO': 1}, 'FEASIBILITY_STUDY_AVAILABLE': {'YES': 0, 'NO': 1, 'NIL': 2}}


In [ ]:
# to be sure that the encoding was succesful
for i in categorical_x_features:
    print(df[i].unique())

[0 1]
[0 1 2 3 4 5 6]
[0 1]
[0 1]
[0 1]
[0 1 2]


In [ ]:
df.head()

,S/N,DATE,NAME_OF_ENTITY,NATURE_OF_BUSINESS,BUSINESS_PROJECT,CONTACT_NAME_PHONE_EMAIL_OF_PROMOTER,PROPOSED_FACILITY_AMOUNT,PURPOSE_OF_FACILITY,VALUE_CHAIN_CATEGORY,NAME_OF_BANK,...,SECURITY_PROPOSED,FEASIBILITY_STUDY_AVAILABLE,HIGHLIGHTS_OF_DISCUSSION,RM_BM_NAME_PHONE_NUMBER,RM_BM_EMAIL,STATUS_UPDATE,CHALLENGES,PROPOSED_NEXT_STEPS,STATE,TARGET
0,1,1/2/2023,COM10001,RICE PROCESSING,0,OB10001,2.000000e+09,WORKING CAPITAL ENHANCEMENT,0,FIDELITY BANK PLC,...,NIL,0,NIL,NIL,NIL,WE AREAWAITING CREDIT COMMITY DECITION ON WEAT...,NONE,AWATING FEEDBACK FROM THE PROSPECT,ADAMAWA,1
1,2,1/2/2023,COM10002,FERTILIZER DISTRIBUTION ACROSS ADAMAWA,0,OB10002,5.000000e+07,WORKING CAPITAL ENHANCEMENT,1,FIDELITY BANK PLC,...,NIL,0,NIL,NIL,NIL,WE ARE CURRENTLY WAITING FOR THE CLIENT TO CHO...,THE CLIENT IS ONLY WILLING TO DISTRIBUTE CRYST...,ONBOARD,ADAMAWA,1
2,3,1/2/2023,COM10003,BULL FEED FINISHING,0,OB10003,9.000000e+06,WORKING CAPITAL ENHANCEMENT,2,STANBIC IBTC,...,EQUITY CONTRIBUTION,0,NIL,NIL,NIL,DOCUMENT YET TO BE PRESENTED,NIL,AWATING FEEDBACK FROM THE PROSPECT,ADAMAWA,1
3,4,1/2/2023,COM10004,BULL FEED FINISHING,0,OB10004,1.000000e+05,WORKING CAPITAL ENHANCEMENT,2,STANBIC IBTC,...,EQUITY CONTRIBUTION,0,NIL,NIL,NIL,DOCUMENT YET TO BE PRESENTED,NIL,AWATING FEEDBACK FROM THE PROSPECT,ADAMAWA,0
4,5,1/2/2023,COM10005,BULL FEED FINISHING,0,OB10005,5.000000e+06,WORKING CAPITAL ENHANCEMENT,2,STANBIC IBTC,...,EQUITY CONTRIBUTION,0,NIL,NIL,NIL,DOCUMENT YET TO BE PRESENTED,NIL,AWATING FEEDBACK FROM THE PROSPECT,ADAMAWA,1


In [ ]:
# splitting data
Y = df['TARGET']
X = df[['BUSINESS_PROJECT', 'VALUE_CHAIN_CATEGORY',
                          'BORROWING_RELATIONSHIP', 'FRESH_LOAN_REQUEST',
                          'REQUEST_SUBMITTED_TO_BANK', 'FEASIBILITY_STUDY_AVAILABLE', 'PROPOSED_FACILITY_AMOUNT']]

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=.2)

## Model Selection and Training

In [ ]:
# using xgboost algorithm
xgboost = xgb.XGBClassifier()

In [ ]:
xgboost_model = xgboost.fit(x_train, y_train)

In [ ]:
xgb_prediction = xgboost_model.predict(x_test)

In [ ]:
accuracy_score(xgb_prediction, y_test)

0.8163265306122449

In [ ]:
# saving the model
joblib.dump(xgboost_model, 'xgboost.joblib')

['xgboost.joblib']

In [ ]:
# loading the model
loaded_model = joblib.load('xgboost.joblib')

In [ ]:
loaded_model.predict(x_test)

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0])

In [ ]:
# using catboost algorithm

In [ ]:
catboost = CatBoostClassifier()

In [ ]:
catboost_model = catboost.fit(x_train, y_train)

Learning rate set to 0.005115
0:	learn: 0.6868552	total: 48.1ms	remaining: 48s
1:	learn: 0.6802886	total: 48.8ms	remaining: 24.4s
2:	learn: 0.6761408	total: 49.7ms	remaining: 16.5s
3:	learn: 0.6701067	total: 50.4ms	remaining: 12.5s
4:	learn: 0.6650490	total: 51.2ms	remaining: 10.2s
5:	learn: 0.6604058	total: 52.2ms	remaining: 8.64s
6:	learn: 0.6549036	total: 52.9ms	remaining: 7.5s
7:	learn: 0.6495795	total: 53.5ms	remaining: 6.63s
8:	learn: 0.6447565	total: 54.3ms	remaining: 5.98s
9:	learn: 0.6406697	total: 55.6ms	remaining: 5.5s
10:	learn: 0.6379284	total: 56.1ms	remaining: 5.05s
11:	learn: 0.6322495	total: 56.7ms	remaining: 4.67s
12:	learn: 0.6276812	total: 57.6ms	remaining: 4.38s
13:	learn: 0.6235582	total: 58.7ms	remaining: 4.13s
14:	learn: 0.6190993	total: 59.8ms	remaining: 3.92s
15:	learn: 0.6150655	total: 61ms	remaining: 3.75s
16:	learn: 0.6125769	total: 61.8ms	remaining: 3.57s
17:	learn: 0.6085344	total: 62.7ms	remaining: 3.42s
18:	learn: 0.6046396	total: 63.4ms	remaining: 3.27

In [ ]:
catboost_prediction = catboost_model.predict(x_test)

In [ ]:
accuracy_score(catboost_prediction, y_test)

0.7959183673469388

In [ ]:
joblib.dump(xgboost_model, 'catboost.joblib')

['catboost.joblib']